In [0]:
# Connecting notebook with Storage Account

# spark.conf.set(
# "fs.azure.account.key.<storage-account>.dfs.core.windows.net",
# "<storage account key>") 

In [0]:

# # Define the path to the ADLS Gen2 storage container
# adls_path = "abfss://<target-container>@<storage-account>.dfs.core.windows.net/"

# # Use dbutils.fs.ls to list the contents of the specified path
# contents = dbutils.fs.ls(adls_path)


###Importing Libraries and Loading Data

In [0]:
# Import necessary libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, desc, isnull, mean, when, explode
from pyspark.sql import functions as F


# Initialize the Spark session
spark = SparkSession.builder.appName("Book Recommendation System").getOrCreate()

In [0]:
# Define the path to the storage container
adls_path = " "

# Load Books dataset 
books_df = spark.read.csv(adls_path + 'Books.csv', header=True, inferSchema=True)

# Load Ratings dataset 
ratings_df = spark.read.csv(adls_path + 'Ratings.csv', header=True, inferSchema=True)

# Load Users dataset 
users_df = spark.read.csv(adls_path + 'Users.csv', header=True, inferSchema=True)

###Basic Statistics

In [0]:
# Display basic statistics for the Books dataset
print("Books dataset statistics:")
books_df.describe().show()

# Display basic statistics for the Ratings dataset
print("Ratings dataset statistics:")
ratings_df.describe().show()

# Display basic statistics for the Users dataset
print("Users dataset statistics:")
users_df.describe().show()


Books dataset statistics:
+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|summary|                ISBN|          Book-Title|         Book-Author| Year-Of-Publication|           Publisher|         Image-URL-S|         Image-URL-M|         Image-URL-L|
+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|  count|              271360|              271360|              271359|              271360|              271358|              271360|              271360|              271357|
|   mean|1.0412234356977516E9|            Infinity|              2001.0|   1959.754171535147|  3765.6153846153848|             1992.25|              1984.5|             1988.25|
| stddev|1.4877142833957233E9|                -NaN|  1.7320508075688772|   258.01944

###Handling Missing Values

In [0]:
# Calculate missing values in each column of the Books dataset
print("Missing values in Books dataset:")
books_df.select([count(when(isnull(c), c)).alias(c) for c in books_df.columns]).show()

# Calculate missing values in each column of the Ratings dataset
print("Missing values in Ratings dataset:")
ratings_df.select([count(when(isnull(c), c)).alias(c) for c in ratings_df.columns]).show()

# Calculate missing values in each column of the Users dataset
print("Missing values in Users dataset:")
users_df.select([count(when(isnull(c), c)).alias(c) for c in users_df.columns]).show()

Missing values in Books dataset:
+----+----------+-----------+-------------------+---------+-----------+-----------+-----------+
|ISBN|Book-Title|Book-Author|Year-Of-Publication|Publisher|Image-URL-S|Image-URL-M|Image-URL-L|
+----+----------+-----------+-------------------+---------+-----------+-----------+-----------+
|   0|         0|          1|                  0|        2|          0|          0|          3|
+----+----------+-----------+-------------------+---------+-----------+-----------+-----------+

Missing values in Ratings dataset:
+-------+----+-----------+
|User-ID|ISBN|Book-Rating|
+-------+----+-----------+
|      0|   0|          0|
+-------+----+-----------+

Missing values in Users dataset:
+-------+--------+------+
|User-ID|Location|   Age|
+-------+--------+------+
|      1|       0|110518|
+-------+--------+------+



###Missing Values in Book Dataset

In [0]:
# Removing rows with missing ISBNs, Book-Title, Book-Author, or Publisher
books_df = books_df.dropna(subset=["ISBN", "Book-Title", "Book-Author", "Publisher"])

# Fill missing Year-Of-Publication with a placeholder value such as -1
books_df = books_df.fillna({"Year-Of-Publication": -1})

# Fill missing image URLs with a default URL or leave them as is
# Example: books_df = books_df.fillna({"Image-URL-S": "default_url", "Image-URL-M": "default_url", "Image-URL-L": "default_url"})

print("Handled missing values in Books dataset.")

Handled missing values in Books dataset.


###Missing Values in the Ratings Dataset

In [0]:
# Removing rows with missing User-ID, ISBN, or Book-Rating
ratings_df = ratings_df.dropna(subset=["User-ID", "ISBN", "Book-Rating"])

print("Handled missing values in Ratings dataset.")


Handled missing values in Ratings dataset.


###Missing Values in Users Dataset


In [0]:
# Removing rows with missing User-ID
users_df = users_df.dropna(subset=["User-ID"])

# Replace missing location with a placeholder value, such as "Unknown"
users_df = users_df.fillna({"Location": "Unknown"})

# Replace missing Age with the mean age (you can also use median)
mean_age = users_df.select(F.mean("Age")).first()[0]
users_df = users_df.fillna({"Age": mean_age})

print("Handled missing values in Users dataset.")


Handled missing values in Users dataset.


###Checking Dataset after cleaning 

In [0]:
# Calculate missing values in each column of the Books dataset
print("Missing values in Books dataset:")
books_df.select([count(when(isnull(c), c)).alias(c) for c in books_df.columns]).show()

# Calculate missing values in each column of the Ratings dataset
print("Missing values in Ratings dataset:")
ratings_df.select([count(when(isnull(c), c)).alias(c) for c in ratings_df.columns]).show()

# Calculate missing values in each column of the Users dataset
print("Missing values in Users dataset:")
users_df.select([count(when(isnull(c), c)).alias(c) for c in users_df.columns]).show()



Missing values in Books dataset:
+----+----------+-----------+-------------------+---------+-----------+-----------+-----------+
|ISBN|Book-Title|Book-Author|Year-Of-Publication|Publisher|Image-URL-S|Image-URL-M|Image-URL-L|
+----+----------+-----------+-------------------+---------+-----------+-----------+-----------+
|   0|         0|          0|                  0|        0|          0|          0|          3|
+----+----------+-----------+-------------------+---------+-----------+-----------+-----------+

Missing values in Ratings dataset:
+-------+----+-----------+
|User-ID|ISBN|Book-Rating|
+-------+----+-----------+
|      0|   0|          0|
+-------+----+-----------+

Missing values in Users dataset:
+-------+--------+---+
|User-ID|Location|Age|
+-------+--------+---+
|      0|       0|  0|
+-------+--------+---+



###Checking the Dataset Size

In [0]:
# Calculate the number of rows and columns in the Books dataset
print("Books dataset size:")
print((books_df.count(), len(books_df.columns)))

# Calculate the number of rows and columns in the Ratings dataset
print("Ratings dataset size:")
print((ratings_df.count(), len(ratings_df.columns)))

# Calculate the number of rows and columns in the Users dataset
print((users_df.count(), len(users_df.columns)))

Books dataset size:
(271357, 8)
Ratings dataset size:
(1149780, 3)
(278858, 3)


###Books Data Analysis

In [0]:
# Analyze the distribution of Year-Of-Publication
print("Year-Of-Publication distribution:")
books_df.groupBy("Year-Of-Publication").count().orderBy("Year-Of-Publication").show()

# Identify the top publishers based on the number of books published
print("Top Publishers:")
books_df.groupBy("Publisher").count().orderBy(desc("count")).show(10)

# Analyze the length of book titles and summarize the statistics
print("Length of book titles:")
books_df.withColumn("title_length", F.length(col("Book-Title"))).describe("title_length").show()


Year-Of-Publication distribution:
+--------------------+-----+
| Year-Of-Publication|count|
+--------------------+-----+
|                1961|    1|
|              D.S.O.|    1|
| Internet (La cou...|    1|
|          Mary Noble|    1|
|    Relationships"""|    1|
| Slimmer--And How...|    1|
| \""The School fo...|    1|
| acht unpaginiert...|    1|
| and Anti-Environ...|    1|
| and Box-Office M...|    1|
| and Murder (St. ...|    1|
| and Musicians at...|    1|
|           and River|    1|
|         and Text"""|    1|
| and \""I\"" Is f...|    1|
| and tasty! : com...|    1|
| samtaler med PÃ¤...|    1|
|"Del \Irish"" Mea...|    1|
|                   0| 4618|
|                1376|    1|
+--------------------+-----+
only showing top 20 rows

Top Publishers:
+--------------------+-----+
|           Publisher|count|
+--------------------+-----+
|           Harlequin| 7533|
|          Silhouette| 4220|
|              Pocket| 3905|
|    Ballantine Books| 3782|
|        Bantam Books| 36

### Year of Publication Data Cleaning and Type Conversion

In [0]:
# Clean and convert Year-Of-Publication in Books dataset to integer
def clean_year_of_publication(df):
    df = df.filter((df["Year-Of-Publication"].cast("int").isNotNull()) & 
                   (df["Year-Of-Publication"].cast("int") >= 1800) & 
                   (df["Year-Of-Publication"].cast("int") <= 2023))
    return df.withColumn("Year-Of-Publication", df["Year-Of-Publication"].cast("int"))

books_df = clean_year_of_publication(books_df)
print("Cleaned and converted Year-Of-Publication to integer in Books dataset")

Cleaned and converted Year-Of-Publication to integer in Books dataset


In [0]:
# Analyze the distribution of Year-Of-Publication
print("Year-Of-Publication distribution:")
books_df.groupBy("Year-Of-Publication").count().orderBy("Year-Of-Publication").show()

Year-Of-Publication distribution:
+-------------------+-----+
|Year-Of-Publication|count|
+-------------------+-----+
|               1806|    1|
|               1897|    1|
|               1900|    3|
|               1901|    7|
|               1902|    2|
|               1904|    1|
|               1906|    1|
|               1908|    1|
|               1909|    2|
|               1910|    1|
|               1911|   19|
|               1914|    1|
|               1917|    1|
|               1919|    1|
|               1920|   33|
|               1921|    2|
|               1922|    2|
|               1923|   11|
|               1924|    2|
|               1925|    2|
+-------------------+-----+
only showing top 20 rows



### Duplicate Book Entries

In [0]:
# Check for duplicate entries in the Books dataset
print("Checking for duplicate entries in Books dataset:")
duplicates = books_df.groupBy("ISBN").count().filter(col("count") > 1)
print(f"Number of duplicate entries in Books dataset: {duplicates.count()}")



Checking for duplicate entries in Books dataset:
Number of duplicate entries in Books dataset: 0


###Book Ratings Data Analysis

In [0]:
# Analyze the distribution of ratings
print("Ratings distribution:")
ratings_df.groupBy("Book-Rating").count().orderBy("Book-Rating").show()

# Identify the top-rated books by average rating
print("Top rated books by average rating:")
ratings_df.groupBy("ISBN").agg(mean("Book-Rating").alias("average_rating")).orderBy(desc("average_rating")).show(10)

# Examine user rating activity by counting ratings per user
print("User rating activity:")
ratings_df.groupBy("User-ID").count().orderBy(desc("count")).show(10)

Ratings distribution:
+-----------+------+
|Book-Rating| count|
+-----------+------+
|          0|716109|
|          1|  1770|
|          2|  2759|
|          3|  5996|
|          4|  8904|
|          5| 50974|
|          6| 36924|
|          7| 76457|
|          8|103736|
|          9| 67541|
|         10| 78610|
+-----------+------+

Top rated books by average rating:
+----------+--------------+
|      ISBN|average_rating|
+----------+--------------+
|0690041535|          10.0|
|067088782X|          10.0|
|0425105156|          10.0|
|0525938508|          10.0|
|0945367198|          10.0|
|0859051595|          10.0|
|0394731271|          10.0|
|1886411999|          10.0|
|080482052X|          10.0|
|9707100567|          10.0|
+----------+--------------+
only showing top 10 rows

User rating activity:
+-------+-----+
|User-ID|count|
+-------+-----+
|  11676|13602|
| 198711| 7550|
| 153662| 6109|
|  98391| 5891|
|  35859| 5850|
| 212898| 4785|
| 278418| 4533|
|  76352| 3367|
| 110973| 3

### Users Data Analysis

In [0]:
# Analyze the distribution of user ages
print("User age distribution:")
users_df.groupBy("Age").count().orderBy("Age").show()

# Analyze the geographic distribution of users by location
print("User location distribution:")
users_df.groupBy("Location").count().orderBy(desc("count")).show(10)

# Identify users with missing age values
print("Users with missing age:")
users_df.filter(users_df.Age.isNull()).show()

User age distribution:
+------------------+-----+
|               Age|count|
+------------------+-----+
| \""alexandria\"".|    1|
|         \""n/a\""|    3|
|      \""n/a\"""""|    1|
|        andorra"""|    2|
|      argentina"""|    5|
|            athens|    1|
|      australia"""|    1|
|        austria"""|   13|
|     bangladesh"""|    5|
|        belgium"""|    4|
|         brazil"""|    7|
|          burma"""|    1|
|     cape verde"""|    2|
|          chile"""|    5|
|          china"""|    2|
|       colombia"""|    3|
|     costa rica"""|    2|
|  cote d`ivoire"""|    1|
|        croatia"""|    5|
| czech republic"""|    3|
+------------------+-----+
only showing top 20 rows

User location distribution:
+--------------------+-----+
|            Location|count|
+--------------------+-----+
|london, england, ...| 2506|
|toronto, ontario,...| 2250|
|sydney, new south...| 1744|
|melbourne, victor...| 1708|
|portland, oregon,...| 1629|
|chicago, illinois...| 1526|
|seattle, wash

###Age Data Cleaning and Type Conversion

In [0]:
# Clean and convert Age in Users dataset to integer
def clean_age(df):
    # Filter out records with invalid age values (e.g., strings or ages outside a reasonable range)
    df = df.filter((df["Age"].cast("int").isNotNull()) & 
                   (df["Age"].cast("int") >= 5) & 
                   (df["Age"].cast("int") <= 120))
    # Convert the column to integer type
    return df.withColumn("Age", df["Age"].cast("int"))

users_df = clean_age(users_df)
print("Cleaned and converted Age to integer in Users dataset")

Cleaned and converted Age to integer in Users dataset


In [0]:
# Analyze the distribution of user ages
print("User age distribution:")
users_df.groupBy("Age").count().orderBy("Age").show()


User age distribution:
+---+-----+
|Age|count|
+---+-----+
|  5|   26|
|  6|   18|
|  7|   27|
|  8|   54|
|  9|   62|
| 10|   84|
| 11|  121|
| 12|  192|
| 13|  883|
| 14| 1962|
| 15| 2379|
| 16| 2566|
| 17| 3042|
| 18| 3690|
| 19| 3934|
| 20| 4040|
| 21| 4428|
| 22| 4701|
| 23| 5440|
| 24| 5666|
+---+-----+
only showing top 20 rows



### Join DataFrames

In [0]:
# Display basic statistics and data types for the Books dataset
print("Books dataset statistics and data types:")
print("\nData types in Books dataset:")
print(books_df.dtypes)  # Display data types of columns
print("\nSummary statistics for Books dataset:")
books_df.describe().show()

# Display basic statistics and data types for the Ratings dataset
print("\nRatings dataset statistics and data types:")
print("\nData types in Ratings dataset:")
print(ratings_df.dtypes)  # Display data types of columns
print("\nSummary statistics for Ratings dataset:")
ratings_df.describe().show()

# Display basic statistics and data types for the Users dataset
print("\nUsers dataset statistics and data types:")
print("\nData types in Users dataset:")
print(users_df.dtypes)  # Display data types of columns
print("\nSummary statistics for Users dataset:")
users_df.describe().show()



Books dataset statistics and data types:

Data types in Books dataset:
[('ISBN', 'string'), ('Book-Title', 'string'), ('Book-Author', 'string'), ('Year-Of-Publication', 'int'), ('Publisher', 'string'), ('Image-URL-S', 'string'), ('Image-URL-M', 'string'), ('Image-URL-L', 'string')]

Summary statistics for Books dataset:
+-------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+
|summary|                ISBN|          Book-Title|         Book-Author|Year-Of-Publication|           Publisher|         Image-URL-S|         Image-URL-M|         Image-URL-L|
+-------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+
|  count|              266667|              266667|              266667|             266667|              266667|              266667|             

In [0]:
# Join Books and Ratings data based on ISBN
book_ratings_df = books_df.join(ratings_df, on="ISBN", how="inner")
print("Joined Books and Ratings data:")
book_ratings_df.show(5)

# Join Users and Ratings data based on User-ID
user_ratings_df = ratings_df.join(users_df, on="User-ID", how="inner")
print("Joined Users and Ratings data:")
user_ratings_df.show(5)

# Combine all three datasets using ISBN and User-ID
complete_df = book_ratings_df.join(users_df, on="User-ID", how="inner")
print("Complete data join:")
complete_df.show(5)



Joined Books and Ratings data:
+----------+--------------------+-------------------+-------------------+--------------------+--------------------+--------------------+--------------------+-------+-----------+
|      ISBN|          Book-Title|        Book-Author|Year-Of-Publication|           Publisher|         Image-URL-S|         Image-URL-M|         Image-URL-L|User-ID|Book-Rating|
+----------+--------------------+-------------------+-------------------+--------------------+--------------------+--------------------+--------------------+-------+-----------+
|0671002481|The First Wives C...|   Olivia Goldsmith|               1996|              Pocket|http://images.ama...|http://images.ama...|http://images.ama...| 271593|          0|
|0842329129|Left Behind: A No...|         Tim Lahaye|               1996|Tyndale House Pub...|http://images.ama...|http://images.ama...|http://images.ama...| 274634|          0|
|3423620005|Sofies Welt. Roma...|    Jostein Gaarder|               1999|      

### Complete Dataset Analysis

In [0]:
# Analyze the most popular books based on total number of ratings received
print("Most popular books based on total number of ratings received:")
complete_df.groupBy("ISBN", "Book-Title").count().orderBy(desc("count")).show(10)

# Examine the most active users based on the number of books rated
print("Most active users based on number of books rated:")
complete_df.groupBy("User-ID").count().orderBy(desc("count")).show(10)

# Analyze book ratings by user age and location
print("Book ratings by user age:")
complete_df.groupBy("Age").agg(mean("Book-Rating").alias("avg_rating")).orderBy("Age").show()

print("Book ratings by user location:")
complete_df.groupBy("Location").agg(mean("Book-Rating").alias("avg_rating")).orderBy(desc("avg_rating")).show(10)


Most popular books based on total number of ratings received:
+----------+--------------------+-----+
|      ISBN|          Book-Title|count|
+----------+--------------------+-----+
|0971880107|         Wild Animus| 2496|
|0316666343|The Lovely Bones:...| 1290|
|0385504209|   The Da Vinci Code|  881|
|0060928336|Divine Secrets of...|  730|
|0312195516|The Red Tent (Bes...|  723|
|044023722X|     A Painted House|  645|
|0142001740|The Secret Life o...|  613|
|067976402X|Snow Falling on C...|  611|
|0446672211|Where the Heart I...|  583|
|0671027360| Angels &amp; Demons|  583|
+----------+--------------------+-----+
only showing top 10 rows

Most active users based on number of books rated:
+-------+-----+
|User-ID|count|
+-------+-----+
|  11676|10915|
| 198711| 6356|
| 153662| 5789|
|  98391| 5776|
|  35859| 5615|
| 212898| 4268|
| 278418| 3956|
|  76352| 3310|
| 110973| 2949|
| 235105| 2920|
+-------+-----+
only showing top 10 rows

Book ratings by user age:
+---+------------------+
|

### Identify Outliers

In [0]:
# Identify outliers in User Age
print("Outliers in User Age:")
complete_df.select("Age").summary().show()

# Identify outliers in Book Rating
print("Outliers in Book Rating:")
complete_df.select("Book-Rating").summary().show()



Outliers in User Age:
+-------+------------------+
|summary|               Age|
+-------+------------------+
|  count|           1014122|
|   mean| 36.38844931872102|
| stddev|10.875290030533929|
|    min|                 5|
|    25%|                31|
|    50%|                34|
|    75%|                41|
|    max|               118|
+-------+------------------+

Outliers in Book Rating:
+-------+------------------+
|summary|       Book-Rating|
+-------+------------------+
|  count|           1014122|
|   mean|  2.83427634939386|
| stddev|3.8530030679781424|
|    min|                 0|
|    25%|                 0|
|    50%|                 0|
|    75%|                 7|
|    max|                10|
+-------+------------------+



### Correlations

In [0]:
# Compute correlation between age and book ratings
print("Correlation between age and book ratings:")
correlation = complete_df.select(F.corr("Age", "Book-Rating")).collect()[0][0]
print(f"Correlation: {correlation}")



Correlation between age and book ratings:
Correlation: -0.022934210623970143


### User Demographics

In [0]:
# Compare book ratings given by different age groups
print("Book ratings by age groups:")

# Group ages by decades using floor division
age_groups = complete_df.withColumn("age_group", F.floor(col("Age") / 10) * 10)

# Calculate the average rating for each age group
average_ratings_by_age_group = age_groups.groupBy("age_group").agg(F.mean("Book-Rating").alias("avg_rating")).orderBy("age_group")

# Display the results
average_ratings_by_age_group.show()

# Insight:
# Grouping user age into categories (e.g., decades) allows for more meaningful comparisons of book ratings.
# This can reveal trends or preferences across different age groups.
# Consider using other age groupings (e.g., age bins) depending on the data distribution and specific analysis goals.

Book ratings by age groups:
+---------+------------------+
|age_group|        avg_rating|
+---------+------------------+
|        0|1.9832285115303983|
|       10|3.6899890976720324|
|       20|  2.84975523495288|
|       30|  2.82758246043587|
|       40|2.7265694792776456|
|       50|2.8735996526270084|
|       60| 2.349170865169664|
|       70| 4.473752012882447|
|       80|3.0348675034867503|
|       90|2.9692307692307693|
|      100|2.5444444444444443|
|      110|1.3027806385169929|
+---------+------------------+



### Publisher Analysis

In [0]:
# Explore which publishers receive the highest average book ratings
print("Publishers with highest average book ratings:")
publisher_ratings = complete_df.groupBy("Publisher").agg(mean("Book-Rating").alias("avg_rating"))
top_publishers = publisher_ratings.orderBy(desc("avg_rating")).show(20)


Publishers with highest average book ratings:
+--------------------+----------+
|           Publisher|avg_rating|
+--------------------+----------+
|Colour Energy Cor...|      10.0|
|Macdonald and Jane's|      10.0|
|Scribes Valley Pu...|      10.0|
|        Bourget Bros|      10.0|
|           D. Spears|      10.0|
|         Veritas Pub|      10.0|
|          Veritas Pr|      10.0|
|Unity School of C...|      10.0|
|         Hermetic Pr|      10.0|
|         Sopris West|      10.0|
| Indigo Publications|      10.0|
|           Jugglebug|      10.0|
|    Providence Press|      10.0|
|           Sattre Pr|      10.0|
|          I.P.A.C.S.|      10.0|
|         Paper Tiger|      10.0|
|    Arden Publishers|      10.0|
|Malice Aforethoug...|      10.0|
|  Wallbuilders Press|      10.0|
|Helwig Industries...|      10.0|
+--------------------+----------+
only showing top 20 rows



### Books with Highest/ Lowest Ratings

In [0]:
# Identify books with the highest average ratings
print("Books with highest average ratings:")
complete_df.groupBy("ISBN", "Book-Title").agg(mean("Book-Rating").alias("avg_rating")).orderBy(desc("avg_rating")).show(10)

# Identify books with the lowest average ratings
print("Books with lowest average ratings:")
complete_df.groupBy("ISBN", "Book-Title").agg(mean("Book-Rating").alias("avg_rating")).orderBy("avg_rating").show(10)


Books with highest average ratings:
+----------+--------------------+----------+
|      ISBN|          Book-Title|avg_rating|
+----------+--------------------+----------+
|1886411999|Absolute OpenBSD:...|      10.0|
|0525938508|Life on the Color...|      10.0|
|0945367198|Freaks, Geeks and...|      10.0|
|0425105156|Accent on Desire ...|      10.0|
|080482052X|Blue and White Japan|      10.0|
|067088782X|Woman's Day Craft...|      10.0|
|0690041535|Yankee Doodle's L...|      10.0|
|156987512X|The Town Mouse an...|      10.0|
|0394731271|Western Forests (...|      10.0|
|0859051595|Yammatji: Aborigi...|      10.0|
+----------+--------------------+----------+
only showing top 10 rows

Books with lowest average ratings:
+----------+--------------------+----------+
|      ISBN|          Book-Title|avg_rating|
+----------+--------------------+----------+
|0571193307|Aunt Margaret's L...|       0.0|
|0373164300|Charity'S Angel (...|       0.0|
|0373163177|Of Dreams And Mag...|       0.0|
|03

### Unique Values

In [0]:
# Calculate the number of unique ISBN values in the Books dataset
print("Unique ISBN values:")
print(books_df.select("ISBN").distinct().count())

# Calculate the number of unique User-ID values in the Users dataset
print("Unique User-ID values:")
print(users_df.select("User-ID").distinct().count())


Unique ISBN values:
266667
Unique User-ID values:
277233


### Frequent Book- User Pairs

In [0]:
# Analyze which books and users appear together most frequently
print("Frequent book-user pairs:")
# Group by 'User-ID' and 'ISBN' to find the frequency of each book-user pair
frequent_pairs = complete_df.groupBy("User-ID", "ISBN").count()

# Order the pairs in ascending order of count
frequent_pairs.orderBy("count").show(10)


Frequent book-user pairs:
+-------+----------+-----+
|User-ID|      ISBN|count|
+-------+----------+-----+
| 277375|3548603203|    1|
| 276936|0749317256|    1|
| 276733|2080674722|    1|
| 276798|3548603203|    1|
| 276964|0515131520|    1|
| 276798|3499134004|    1|
| 277427|0425087859|    1|
| 277378|0767906373|    1|
| 277157|0441005470|    1|
| 276904|0738205737|    1|
+-------+----------+-----+
only showing top 10 rows



### Recommendation 

In [0]:
##### Recommendation Based on User Age and Book Ratings




# Ask for user ID input to recommend books
user_id = int(input("Enter the user ID for recommendations: "))

# Get the user's age from the Users dataset
user_age_row = users_df.filter(users_df["User-ID"] == user_id).select("Age").collect()

# Check if the user was found in the dataset
if user_age_row:
    user_age = user_age_row[0]["Age"]

    # Create an age_group column by grouping ages by decades
    age_group = (int(user_age) // 10) * 10

    # Filter the complete dataset for the specified age group
    age_group_data = complete_df.filter((complete_df["Age"] >= age_group) & (complete_df["Age"] < age_group + 10))

    # Group by ISBN and Book-Title and calculate the average book rating for the user's age group
    recommendation_candidates = age_group_data.groupBy("ISBN", "Book-Title").agg(mean("Book-Rating").alias("avg_rating")).orderBy(desc("avg_rating"))

    # Display the top recommendation candidates for the user's age group
    print(f"Top recommendation candidates for User ID {user_id} (Age Group: {age_group}s):")
    recommendation_candidates.show(10)


else:
    print(f"User ID {user_id} not found or invalid age.")

Enter the user ID for recommendations:  123

Top recommendation candidates for User ID 123 (Age Group: 20s):
+----------+--------------------+----------+
|      ISBN|          Book-Title|avg_rating|
+----------+--------------------+----------+
|0060923288|Raising Your Spir...|      10.0|
|0060173890|Frankly Scarlett,...|      10.0|
|8478445137|        de Profundis|      10.0|
|1558506454|Museum of Science...|      10.0|
|1593080409|The Complete Sher...|      10.0|
|0385260458|Anthills of the S...|      10.0|
|0811201856|A Season in Hell ...|      10.0|
|0140309128|Danny, the Champi...|      10.0|
|0890877254|The Totally Garli...|      10.0|
|0440226236|       The Death Pit|      10.0|
+----------+--------------------+----------+
only showing top 10 rows



In [0]:
##### Trending and Popular Books


# Find books with the most recent increase in popularity
from pyspark.sql.window import Window
trending_books = complete_df.groupBy("ISBN", "Book-Title").agg(count("User-ID").alias("count"))
trending_books = trending_books.withColumn("rank", F.row_number().over(Window.orderBy(desc("count"))))
print("Top trending books:")
trending_books.filter(trending_books["rank"] <= 10).show()

Top trending books:
+----------+--------------------+-----+----+
|      ISBN|          Book-Title|count|rank|
+----------+--------------------+-----+----+
|0971880107|         Wild Animus| 2496|   1|
|0316666343|The Lovely Bones:...| 1290|   2|
|0385504209|   The Da Vinci Code|  881|   3|
|0060928336|Divine Secrets of...|  730|   4|
|0312195516|The Red Tent (Bes...|  723|   5|
|044023722X|     A Painted House|  645|   6|
|0142001740|The Secret Life o...|  613|   7|
|067976402X|Snow Falling on C...|  611|   8|
|0446672211|Where the Heart I...|  583|   9|
|0671027360| Angels &amp; Demons|  583|  10|
+----------+--------------------+-----+----+



#### Recommendation System

In [0]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

# Convert ISBN column from string to numeric values
indexer = StringIndexer(inputCol="ISBN", outputCol="ISBNIndex")
indexed_df = indexer.fit(complete_df).transform(complete_df)

# Prepare the data
indexed_df = indexed_df.filter(indexed_df["Book-Rating"] > 0)  # Filter out 0 ratings
(training_data, test_data) = indexed_df.randomSplit([0.8, 0.2])  # Split data into training and test sets

# Initialize ALS model
als = ALS(
    maxIter=2,  # Number of iterations to run the algorithm
    regParam=0.1,  # Regularization parameter to avoid overfitting
    userCol="User-ID",  # Column representing users
    itemCol="ISBNIndex",  # Column representing items (books) after indexing
    ratingCol="Book-Rating",  # Column representing ratings
    coldStartStrategy="drop"  # Strategy to handle cold start (unknown users/items)
)



In [0]:
# Train the ALS model
model = als.fit(training_data)

# Evaluate the model using RMSE
evaluator = RegressionEvaluator(
    metricName="rmse",
    labelCol="Book-Rating",
    predictionCol="prediction"
)
predictions = model.transform(test_data)
rmse = evaluator.evaluate(predictions)
print(f"Root Mean Square Error (RMSE): {rmse}")

# Make book recommendations for each user
print("Top 10 book recommendations for each user:")
user_recommendations = model.recommendForAllUsers(10)
user_recommendations.show(5)


Root Mean Square Error (RMSE): 8.104666694617759
Top 10 book recommendations for each user:
+-------+--------------------+
|User-ID|     recommendations|
+-------+--------------------+
|     12|[{8196, 12.113528...|
|     16|[{49021, 20.95031...|
|     22|[{5661, 8.119773}...|
|     26|[{7777, 28.627188...|
|     44|[{14725, 8.143021...|
+-------+--------------------+
only showing top 5 rows



In [0]:
# Make book recommendations for each user
user_recommendations = model.recommendForAllUsers(5)  # Recommend 5 books for each user

# Explode the recommendations array to get individual recommendations
exploded_recommendations = user_recommendations \
    .withColumn("recommendation", explode(user_recommendations["recommendations"])) \
    .select(col("User-ID"), col("recommendation.ISBNIndex").alias("ISBNIndex"))

# Join with indexed_df to get the complete book titles based on ISBNIndex
user_recommendations_with_titles = exploded_recommendations \
    .join(indexed_df, "ISBNIndex") \
    .select(exploded_recommendations["User-ID"], indexed_df["Book-Title"])

# Show 5 recommendations of book titles for each user
print("Top 5 book recommendations (titles) for each user:")
user_recommendations_with_titles.show(truncate=False)


Top 5 book recommendations (titles) for each user:
+-------+-------------------------------------+
|User-ID|Book-Title                           |
+-------+-------------------------------------+
|252356 |Harry Potter und der Stein der Weisen|
|216057 |Harry Potter und der Stein der Weisen|
|94781  |Harry Potter und der Stein der Weisen|
|215829 |Harry Potter und der Stein der Weisen|
|125878 |Harry Potter und der Stein der Weisen|
|109620 |Harry Potter und der Stein der Weisen|
|74272  |Harry Potter und der Stein der Weisen|
|274831 |Harry Potter und der Stein der Weisen|
|248607 |Harry Potter und der Stein der Weisen|
|235055 |Harry Potter und der Stein der Weisen|
|209147 |Harry Potter und der Stein der Weisen|
|26143  |Harry Potter und der Stein der Weisen|
|261941 |Blackout                             |
|228508 |Blackout                             |
|219153 |Blackout                             |
|216932 |Blackout                             |
|208590 |Blackout                    